In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('sudoku.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000000 entries, 0 to 8999999
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   puzzle    object
 1   solution  object
dtypes: object(2)
memory usage: 137.3+ MB


In [4]:
df.head()

,puzzle,solution
0,0700000430400096108006349000940520003584600200...,6795182435437296188216349577943521863584617292...
1,3010865040465210705000000014008000020803479000...,3719865248465213795924738614638197522853479167...
2,0483015603600080909106700030200009355090102006...,7483915623652487919126754834217869355894132766...
3,0083170000042051090000400703271609049014500000...,2983176457642851391539462783271689549814537266...
4,0408906300001368208007405190004670524500207002...,1428956379751368248367425193984671524513287962...


In [5]:
df["puzzle"].dtype

dtype('O')

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np

In [7]:
X, Y = df['puzzle'], df['solution']

In [8]:
#X_reshaped = X.apply(lambda i: np.array([int(j) for j in i]).reshape((9,9,1)))

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=100)

#Y_reshaped = Y.apply(lambda i: np.array([int(j) for j in i]).reshape((81,1)))

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [9]:
feat = []
label = []

for i in X:
    x = np.array([int(j) for j in i]).reshape((9,9,1))
    feat.append(x)

feat = np.array(feat)
feat = feat/9
feat -= .5    

for i in Y:
    x = np.array([int(j) for j in i]).reshape((81,1)) - 1
    label.append(x)   

label = np.array(label)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.20, random_state=100)

In [11]:
epochs = 20

In [12]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(label.shape)

(7200000, 9, 9, 1)
(1800000, 9, 9, 1)
(7200000, 81, 1)
(1800000, 81, 1)
(9000000, 81, 1)


y_train = y_train.reshape((-1,9))

In [13]:
# convert class labels (from digits) to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, 9)
y_test = tf.keras.utils.to_categorical(y_test, 9)
print(y_train.shape)
print(y_test.shape)

(7200000, 81, 9)
(1800000, 81, 9)


In [14]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape

In [15]:
model = keras.models.Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

In [16]:
# metric is accuracy
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten (Flatten)            (None, 10368)             0         
_________________________________________________________________
dense (Dense)                (None, 729)               7

In [18]:
history = model.fit(x_train, y_train,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
  4434/225000 [..............................] - ETA: 5:38:09 - loss: 2.3594 - accuracy: 0.1139

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()